In [ ]:
import lightning as L
import torch
import torch.nn as nn
import torch.nn.functional as F

from pathlib import Path
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint
import json
import os

from train import RawNet2Trainer, AASISTTrainer, W2VTrainer
from data import EchoFakeModule, ASVspoof2019, IntheWild, ASVspoofEval, WaveFake

In [2]:
import glob

sampled_asvspoof = glob.glob(
    "/data1/zt/ReplayDeepfake/archives/sampled_asvspoof/*.flac"
)
recorded_asvspoof = glob.glob(
    "/data1/zt/ReplayDeepfake/archives/asvspoof_recorded/*.flac"
)

metadata = "/data1/datasets/ASVspoof2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt"
label_dict = {}
with open(metadata, "r") as f:
    lines = f.readlines()
    for line in lines:
        _, utt_id, _, _, label = line.split()
        label_dict[utt_id] = 0 if label == "spoof" else 1

In [7]:
# load model
# ckpt = "exp/W2V-Linear-ASV/version_0/checkpoints/best.ckpt"
# with open("configs/w2v.json") as f:
#     config = json.load(f)
# model = W2VTrainer.load_from_checkpoint(ckpt, config=config).model
# model.eval()
# model = model.to("cuda")

from models import AASIST

model = AASIST(
    {
        "architecture": "AASIST",
        "nb_samp": 64600,
        "first_conv": 128,
        "filts": [70, [1, 32], [32, 32], [32, 64], [64, 64]],
        "gat_dims": [64, 32],
        "pool_ratios": [0.5, 0.7, 0.5, 0.5],
        "temperatures": [2.0, 2.0, 100.0, 100.0],
    },
    num_classes=2,
)
states = torch.load("models/weights/AASIST.pth")
model.load_state_dict(states)
model.eval()
model = model.to("cuda")

/tmp/ipykernel_107650/2511601282.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  states = torch.load("models/weights/AASIST.pth")


In [8]:
import librosa
import numpy as np
from tqdm import tqdm


def pad(x, max_len=64000):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len) + 1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x


labels = []
preds = []

for path in tqdm(sampled_asvspoof):
    utt_id = os.path.basename(path).split(".")[0]
    y = label_dict[utt_id]

    x, _ = librosa.load(path, sr=16000)
    x = torch.Tensor(pad(x, max_len=64000)).unsqueeze(0).to("cuda")

    _, out, *_ = model(x)
    pred = torch.argmax(out, dim=1).cpu().numpy()[0]

    labels.append(y)
    preds.append(pred)

100%|██████████| 1400/1400 [00:07<00:00, 198.76it/s]


In [9]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
true_negative_rate = tn / (tn + fp)
print(tn, fp, fn, tp)
print("True Negative Rate:", true_negative_rate)

1238 62 0 100
True Negative Rate: 0.9523076923076923


In [10]:
labels = []
preds = []

for path in tqdm(recorded_asvspoof):
    utt_id = os.path.basename(path).split(".")[0]
    y = label_dict[utt_id]

    x, _ = librosa.load(path, sr=16000)
    x = torch.Tensor(pad(x, max_len=64000)).unsqueeze(0).to("cuda")

    _, out, *_ = model(x)
    pred = torch.argmax(out, dim=1).cpu().numpy()[0]

    labels.append(y)
    preds.append(pred)

tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
true_negative_rate = tn / (tn + fp)
print(tn, fp, fn, tp)
print("True Negative Rate:", true_negative_rate)

100%|██████████| 1400/1400 [00:07<00:00, 179.42it/s]

139 1161 0 100
True Negative Rate: 0.10692307692307693
